In [ ]:
# CUDA: Let's check that Nvidia CUDA drivers are already pre-installed and which version is it.
!/usr/local/cuda/bin/nvcc --version

In [ ]:
!ls /usr/bin | grep nvidia
!echo $PATH

In [ ]:
!nvidia-smi

Add the dataset

In [ ]:
!pip install --upgrade dataset-tools

In [ ]:
import dataset_tools as dtools
dtools.download(dataset='deepNIR Fruit Detection', dst_dir='/content/dataset-ninja/')

Check the structure of this dataset

In [ ]:
!ls /content/dataset-ninja/deepnir-fruit-detection

In [ ]:
import os
import json
import shutil
from sklearn.model_selection import train_test_split

# Define paths
train_path = "/content/dataset-ninja/deepnir-fruit-detection/train"
val_path = "/content/dataset-ninja/deepnir-fruit-detection/valid"
test_path = "/content/dataset-ninja/deepnir-fruit-detection/test"

# List the contents of the img and ann directories
train_img_files = os.listdir(os.path.join(train_path, "img"))
train_ann_files = os.listdir(os.path.join(train_path, "ann"))
val_img_files = os.listdir(os.path.join(val_path, "img"))
val_ann_files = os.listdir(os.path.join(val_path, "ann"))
test_img_files = os.listdir(os.path.join(test_path, "img"))
test_ann_files = os.listdir(os.path.join(test_path, "ann"))

print("Number of train images:", len(train_img_files))
print("Number of train annotations:", len(train_ann_files))
print("Number of val images:", len(val_img_files))
print("Number of val annotations:", len(val_ann_files))
print("Number of test images:", len(test_img_files))
print("Number of test annotations:", len(test_ann_files))

# Create directories if not exist
os.makedirs(os.path.join(train_path, "img"), exist_ok=True)
os.makedirs(os.path.join(train_path, "ann"), exist_ok=True)
os.makedirs(os.path.join(val_path, "img"), exist_ok=True)
os.makedirs(os.path.join(val_path, "ann"), exist_ok=True)
os.makedirs(os.path.join(test_path, "img"), exist_ok=True)
os.makedirs(os.path.join(test_path, "ann"), exist_ok=True)

In [ ]:
# discover the "img" and "ann"
# Print the first five files in each directory
print("First five files in 'img':", train_img_files[:5])
print("First five files in 'ann':", train_ann_files[:5])

##if training with only wheat positives, run the following section

We found that it's not sorted, so we need to pair them up manually: ann name(.jpg) = file name.jpg(.json)

Also filter to have only bounding boxes for the wheat class

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

def filter_bounding_boxes(img_dir, ann_dir, visualize=False):
    """
    Filters images and annotations to include only those with bounding boxes
    for the specified class.

    Args:
        img_dir (str): Directory containing image files.
        ann_dir (str): Directory containing annotation files.
        visualize (bool): If True, displays bounding boxes on the image.

    Returns:
        list: Filtered image file paths.
        list: Filtered annotation file paths.
    """
    filtered_imgs = []
    filtered_anns = []

    for img_file in sorted(os.listdir(img_dir)):
        img_path = os.path.join(img_dir, img_file)
        ann_file = img_file + ".json"  # Match image file
        ann_path = os.path.join(ann_dir, ann_file)

        with open(ann_path, 'r') as f:
            annotation = json.load(f)

        # Check if the annotation contains at least one bounding box for the class
        contains_class = any(
            obj["classTitle"] == "wheat" and obj["geometryType"] == "rectangle"
            for obj in annotation.get("objects", [])
        )

        if contains_class:
            filtered_imgs.append(img_path)
            filtered_anns.append(ann_path)

            # Visualization (if enabled)
            if visualize:
                print(f"Visualizing: {img_path}")
                bounding_boxes = [
                    obj["points"]["exterior"]
                    for obj in annotation["objects"]
                    if obj["classTitle"] == "wheat" and obj["geometryType"] == "rectangle"
                ]

                # Open and display the image with bounding boxes
                img = Image.open(img_path).convert("RGB")
                fig, ax = plt.subplots(1, 1, figsize=(10, 10))
                ax.imshow(img)

                # Add bounding boxes
                for bbox in bounding_boxes:
                    x_min, y_min = bbox[0]
                    x_max, y_max = bbox[1]
                    width = x_max - x_min
                    height = y_max - y_min
                    rect = patches.Rectangle((x_min, y_min), width, height, linewidth=2, edgecolor='red', facecolor='none')
                    ax.add_patch(rect)

                ax.set_title(f"Bounding Boxes for {img_path}")
                plt.axis('off')
                plt.show()

            # Extract detected object
            detected_objects = [obj["classTitle"] for obj in annotation.get("objects", [])]
            # print(f"Object Class is: {detected_objects[0]}")

            # Count the number of detected objects
            num_objects = len(detected_objects)
            print(f"{img_file}: Object Class is {detected_objects[0]}; Num detected: {num_objects}")

    print(f"Filtered {len(filtered_imgs)} images and annotations.")

    return filtered_imgs, filtered_anns

set visualize=True if want visualization of images

In [ ]:
train_imgs, train_anns = filter_bounding_boxes(os.path.join(train_path, "img"), os.path.join(train_path, "ann"))
val_imgs, val_anns = filter_bounding_boxes(os.path.join(val_path, "img"), os.path.join(val_path, "ann"))
test_imgs, test_anns = filter_bounding_boxes(os.path.join(test_path, "img"), os.path.join(test_path, "ann"))

In [ ]:
def save_filtered_data(img_paths, ann_paths, img_dest, ann_dest):
    """
    Saves filtered images and annotations into specified directories.

    Args:
        img_paths (list): List of image file paths.
        ann_paths (list): List of annotation file paths.
        img_dest (str): Destination directory for images.
        ann_dest (str): Destination directory for annotations.
    """
    os.makedirs(img_dest, exist_ok=True)
    os.makedirs(ann_dest, exist_ok=True)

    for img_path, ann_path in zip(img_paths, ann_paths):
        shutil.copy(img_path, img_dest)
        shutil.copy(ann_path, ann_dest)

# Paths for saving filtered data
filtered_train_path = "/content/dataset-ninja/wheat/train"
filtered_val_path = "/content/dataset-ninja/wheat/val"
filtered_test_path = "/content/dataset-ninja/wheat/test"

# Save filtered datasets
save_filtered_data(train_imgs, train_anns, os.path.join(filtered_train_path, "img"), os.path.join(filtered_train_path, "ann"))
save_filtered_data(val_imgs, val_anns, os.path.join(filtered_val_path, "img"), os.path.join(filtered_val_path, "ann"))
save_filtered_data(test_imgs, test_anns, os.path.join(filtered_test_path, "img"), os.path.join(filtered_test_path, "ann"))

# Final counts
print(f"Filtered Training set: {len(train_imgs)} images, {len(train_anns)} annotations")
print(f"Filtered Validation set: {len(val_imgs)} images, {len(val_anns)} annotations")
print(f"Filtered Test set: {len(test_imgs)} images, {len(test_anns)} annotations")

##YOLO

In [ ]:
import os

def convert_to_yolo_format(ann_path, img_shape, output_path):
    """
    Convert annotation JSON to YOLO format and save to a .txt file.

    Args:
        ann_path (str): Path to the annotation JSON.
        img_shape (tuple): Shape of the image (width, height).
        output_path (str): Path to save the YOLO formatted annotation.
    """
    with open(ann_path, 'r') as f:
        annotation = json.load(f)

    height, width = img_shape
    with open(output_path, 'w') as f:
        for obj in annotation.get("objects", []):
            if obj["classTitle"] == "wheat"  and obj["geometryType"] == "rectangle":
                # Normalize bounding box coordinates
                x_min, y_min = obj["points"]["exterior"][0]
                x_max, y_max = obj["points"]["exterior"][1]
                x_center = (x_min + x_max) / 2 / width
                y_center = (y_min + y_max) / 2 / height
                box_width = (x_max - x_min) / width
                box_height = (y_max - y_min) / height

                # Write to file in YOLO format: <class_id> <x_center> <y_center> <width> <height>
                f.write(f"0 {x_center} {y_center} {box_width} {box_height}\n")

In [ ]:
def convert_dataset(img_dir, ann_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    for img_file in os.listdir(img_dir):
        img_path = os.path.join(img_dir, img_file)
        ann_file = img_file + ".json"
        ann_path = os.path.join(ann_dir, ann_file)
        output_path = os.path.join(output_dir, img_file.replace(".jpg", ".txt"))

        # Get image dimensions
        img = Image.open(img_path)
        img_shape = img.size  # (width, height)

        # Convert annotation
        convert_to_yolo_format(ann_path, img_shape, output_path)

convert_dataset(
    img_dir=os.path.join(filtered_train_path, "img"),
    ann_dir=os.path.join(filtered_train_path, "ann"),
    output_dir=os.path.join(filtered_train_path, "labels")
)
convert_dataset(
    img_dir=os.path.join(filtered_val_path, "img"),
    ann_dir=os.path.join(filtered_val_path, "ann"),
    output_dir=os.path.join(filtered_val_path, "labels")
)
convert_dataset(
    img_dir=os.path.join(filtered_test_path, "img"),
    ann_dir=os.path.join(filtered_test_path, "ann"),
    output_dir=os.path.join(filtered_test_path, "labels")
)

In [ ]:
# Generate .txt
def generate_image_list(img_dir, output_path):
    with open(output_path, 'w') as f:
        for img_file in os.listdir(img_dir):
            img_path = os.path.join(img_dir, img_file)
            f.write(f"{img_path}\n")

generate_image_list(
    img_dir=os.path.join(filtered_train_path, "img"),
    output_path=os.path.join(filtered_train_path, "train.txt")
)
generate_image_list(
    img_dir=os.path.join(filtered_val_path, "img"),
    output_path=os.path.join(filtered_val_path, "val.txt")
)
generate_image_list(
    img_dir=os.path.join(filtered_test_path, "img"),
    output_path=os.path.join(filtered_test_path, "test.txt")
)

In [ ]:
!mv /content/dataset-ninja/wheat/train/labels/*.txt /content/dataset-ninja/wheat/train/img/
!mv /content/dataset-ninja/wheat/val/labels/*.txt /content/dataset-ninja/wheat/val/img/

##To test with a single file

In [ ]:
# Install darknet (YOLOv4 framework) and other dependencies
!git clone https://github.com/AlexeyAB/darknet.git
%cd darknet

# Build Darknet with GPU and OpenCV support
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

In [ ]:
!make

In [ ]:
os.makedirs("/content/dataset-ninja/wheat/single/", exist_ok=True)

In [ ]:
!cp "/content/dataset-ninja/wheat/train/img/dbfd38b38_jpg.rf.0b2e8a2670b8b16dbd6b7e4dd0e98ff6.jpg" "/content/dataset-ninja/wheat/single/"

In [ ]:
!cp "/content/dataset-ninja/wheat/train/img/dbfd38b38_jpg.rf.0b2e8a2670b8b16dbd6b7e4dd0e98ff6.txt" "/content/dataset-ninja/wheat/single/"

In [ ]:
!echo "/content/dataset-ninja/wheat/single/dbfd38b38_jpg.rf.0b2e8a2670b8b16dbd6b7e4dd0e98ff6.jpg" > /content/dataset-ninja/wheat/single/train.txt

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/yolov4/yolov4.weights

In [ ]:
!echo "wheat" > /content/darknet/data/obj.names

In [ ]:
with open("/content/darknet/obj.data", "w") as f:
    f.write("""classes=1
train=/content/dataset-ninja/wheat/single/train.txt
valid=/content/dataset-ninja/wheat/single/train.txt
names=/content/darknet/data/obj.names
backup=/content/darknet/backup/
""")

In [ ]:
cfg_path = "/content/darknet/cfg/yolov4-custom.cfg"

with open(cfg_path, "r") as f:
    lines = f.readlines()

num_classes = 1
filters = (num_classes + 5) * 3

updated_lines = []
for line in lines:
    if line.strip().startswith("batch="):
        updated_lines.append("batch=64\n")
    elif line.strip().startswith("subdivisions="):
        updated_lines.append("subdivisions=4\n")
    elif line.strip().startswith("max_batches = "):
        updated_lines.append("max_batches=2000\n")
        print(updated_lines)
    elif line.strip().startswith("steps="):
        max_batches = num_classes * 2000
        updated_lines.append(f"steps={int(0.8 * max_batches)},{int(0.9 * max_batches)}\n")
    elif line.strip().startswith("filters="):
        updated_lines.append(f"filters={filters}\n")
    elif line.strip().startswith("classes="):
        updated_lines.append(f"classes={num_classes}\n")
    else:
        updated_lines.append(line)

with open(cfg_path, "w") as f:
    f.writelines(updated_lines)

print(f"Configuration file updated at: {cfg_path}")

In [ ]:
# !./darknet detector map /content/darknet/obj.data /content/darknet/cfg/yolov4-custom.cfg /content/darknet/yolov4.conv.137 -map

!./darknet detector train /content/darknet/obj.data /content/darknet/cfg/yolov4-custom.cfg /content/darknet/yolov4.weights -map

In [ ]:
!./darknet detector test /content/darknet/obj.data /content/darknet/cfg/yolov4-custom.cfg /content/darknet/yolov4.weights -ext_output -dont_show < /content/dataset-ninja/wheat/single/train.txt

In [ ]:
from IPython.display import Image as IPImage, display

display(IPImage(filename='/content/darknet/predictions.jpg'))

##To run on the entire dataset

In [ ]:
# Install darknet (YOLOv4 framework) and other dependencies
!git clone https://github.com/AlexeyAB/darknet.git
%cd darknet

# Build Darknet with GPU and OpenCV support
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

In [ ]:
# make darknet (builds darknet so that you can then use the darknet executable file to run or train object detectors)
!make

In [ ]:
# Display dimensions of the first 10 images
import os
from PIL import Image

img_dir = '/content/dataset-ninja/wheat/train/img'
dimensions = [Image.open(os.path.join(img_dir, f)).size for f in os.listdir(img_dir)]
print(dimensions[:10])

In [ ]:
!echo "wheat" > /content/darknet/data/obj.names

In [ ]:
with open("/content/darknet/obj.data", "w") as f:
    f.write("""classes=1
train=/content/dataset-ninja/wheat/train/train.txt
valid=/content/dataset-ninja/wheat/val/val.txt
names=/content/darknet/data/obj.names
backup=/content/darknet/backup/
""")

In [ ]:
cfg_path = "/content/darknet/cfg/yolov4-custom.cfg"

with open(cfg_path, "r") as f:
    lines = f.readlines()

num_classes = 1
filters = (num_classes + 5) * 3

updated_lines = []
for line in lines:
    if line.strip().startswith("batch="):
        updated_lines.append("batch=64\n")
    elif line.strip().startswith("subdivisions="):
        updated_lines.append("subdivisions=4\n")
    elif line.strip().startswith("max_batches = "):
        updated_lines.append("max_batches=2000\n")
        print(updated_lines)
    elif line.strip().startswith("steps="):
        max_batches = num_classes * 2000
        updated_lines.append(f"steps={int(0.8 * max_batches)},{int(0.9 * max_batches)}\n")
    elif line.strip().startswith("filters="):
        updated_lines.append(f"filters={filters}\n")
    elif line.strip().startswith("classes="):
        updated_lines.append(f"classes={num_classes}\n")
    else:
        updated_lines.append(line)

with open(cfg_path, "w") as f:
    f.writelines(updated_lines)

print(f"Configuration file updated at: {cfg_path}")

In [ ]:
# getting yolov4.conv.137 weights
# !wget https://github.com/AlexeyAB/darknet/releases/download/yolov4/yolov4.conv.137 -P /content/darknet/

# getting yolov4.weights
!wget https://github.com/AlexeyAB/darknet/releases/download/yolov4/yolov4.weights -P /content/darknet/

In [ ]:
# Train YOLOv4
!./darknet detector train /content/darknet/obj.data /content/darknet/cfg/yolov4-custom.cfg yolov4.conv.137 -map -dont_show -saveweights 100

In [ ]:
# for debug - Evaluate mAP

# if completed training:
# !./darknet detector map /content/darknet/obj.data /content/darknet/cfg/yolov4-custom.cfg /content/darknet/backup/yolov4-custom_best.weights

# otherwise:
# !./darknet detector map /content/darknet/obj.data /content/darknet/cfg/yolov4-custom.cfg /content/darknet/backup/yolov4-custom_last.weights

Make Predictions on Test Images, The predictions will be stored in result.json

In [ ]:
# Predict on test images
!./darknet detector test /content/darknet/obj.data /content/darknet/cfg/yolov4-custom.cfg /content/darknet/backup/yolov4-custom_last.weights -ext_output -dont_show -thresh 0.1 -out result.json < /content/dataset-ninja/wheat/test/test.txt

In [ ]:
!cat result.json

##Future measurement steps

In [ ]:
import json
import os

def compute_iou(pred_box, gt_box):
    x_min = max(pred_box[0], gt_box[0])
    y_min = max(pred_box[1], gt_box[1])
    x_max = min(pred_box[2], gt_box[2])
    y_max = min(pred_box[3], gt_box[3])
    inter_area = max(0, x_max - x_min) * max(0, y_max - y_min)
    pred_area = (pred_box[2] - pred_box[0]) * (pred_box[3] - gt_box[1])
    gt_area = (gt_box[2] - gt_box[0]) * (gt_box[3] - gt_box[1])
    return inter_area / (pred_area + gt_area - inter_area + 1e-6)

def load_ground_truths(annotation_dir):
    ground_truths = {}
    for file_name in os.listdir(annotation_dir):
        if file_name.endswith(".json"):
            file_path = os.path.join(annotation_dir, file_name)
            with open(file_path, "r") as f:
                data = json.load(f)
                # Extract bounding boxes in [x_min, y_min, x_max, y_max] format
                boxes = [
                    [obj["x_min"], obj["y_min"], obj["x_max"], obj["y_max"]]
                    for obj in data.get("annotations", [])
                ]
                ground_truths[file_name.replace(".json", ".jpg")] = boxes
    return ground_truths

def calculate_metrics(predictions, ground_truths):
    tp, fp, fn = 0, 0, 0

    for img, preds in predictions.items():
        gt_boxes = ground_truths.get(img, [])
        matched = [False] * len(gt_boxes)  # Track matched ground truths

        for pred in preds:
            iou_scores = [compute_iou(pred, gt) for gt in gt_boxes]
            max_iou = max(iou_scores) if iou_scores else 0
            max_iou_idx = iou_scores.index(max_iou) if iou_scores else -1

            if max_iou >= 0.5 and not matched[max_iou_idx]:
                tp += 1
                matched[max_iou_idx] = True  # Mark this ground truth as matched
            else:
                fp += 1

        fn += sum(1 for m in matched if not m)  # Unmatched ground truths are false negatives

    precision = tp / (tp + fp + 1e-6)
    recall = tp / (tp + fn + 1e-6)
    f1 = 2 * (precision * recall) / (precision + recall + 1e-6)

    return {"precision": precision, "recall": recall, "f1": f1}

# Paths
annotation_dir = "/content/dataset-ninja/wheat/ann/"
results_path = "/content/result.json"

# Load ground truths
ground_truths = load_ground_truths(annotation_dir)

# Load predictions
with open(results_path, "r") as f:
    predictions = json.load(f)

# Calculate metrics
metrics = calculate_metrics(predictions, ground_truths)
print(metrics)